<a href="https://colab.research.google.com/github/alvs210/Alveena.github.io/blob/main/EDGARPoemsChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Edgar, the poetry bot!

Tell it how you feel (ex. I feel sad) to recieve a poem that'll fit the mood.
Feel free to add new poems. He'll tell you how!

If you get sick of him at any time, say "quit"

To begin:
1.   Click "Runtime" in the toolbar above.
2.   Click "Run all"

And chat away! :)

The final poetry dataset including your changes will be automatically downloaded to the browser at the end. Feel free to not save it.





In [ ]:
# @title
!pip install tensorflow keras nltk numpy

In [ ]:
# @title
import json
import requests
import tempfile

# Replace with the raw URL of your JSON file on GitHub
file_path = 'https://raw.githubusercontent.com/alvs210/PoetryChatter/main/PoetryIntents1234.json'

# Fetch the JSON file from GitHub
response = requests.get(file_path)

# Load the JSON data
dataset = json.loads(response.text)

with tempfile.NamedTemporaryFile(mode='w+', suffix='.json', delete=False) as f:
    json.dump(dataset, f, indent=4)
    temp_file_path = f.name
    print(f"Temporary JSON file created: {temp_file_path}")

# ... (Now you can use 'temp_file_path' to access the saved JSON file)


Temporary JSON file created: /tmp/tmp1683awh1.json


In [ ]:
# @title
import nltk
import tensorflow as tf
import json
import numpy as np
import pandas as pd
import pickle
import random


In [ ]:
# @title
nltk.download('punkt')
nltk.download('wordnet')


words = []
documents = []
tags = []

for data in dataset["intents"]:
    for pattern in data['patterns']:
        # Tokenize each word using nltk library
        # Tokenize -> Break the Sentence into words
        word = nltk.word_tokenize(pattern)
        # print(word)
        words.extend(word)
        # Group the Tokenized Word and the Tag
        documents.append((word, data['tag']))
        # Add Tags from the dataset
        if data['tag'] not in tags:
            tags.append(data['tag'])

# Lemmatize the data
# Lemmatize -> Grouping the differents words into one word
lemmatizer = nltk.stem.WordNetLemmatizer()
# Lemmatize the unique word in the words
words = [lemmatizer.lemmatize(word.lower()) for word in words]
words = sorted(list(set(words)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# @title
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(tags, open('tags.pkl', 'wb'))

In [ ]:
# @title
training = []

for docs in documents:
    bag_of_words = np.zeros(shape=(len(words)), dtype=np.float32)
    tag_pattern = np.zeros(shape=(len(tags)), dtype=np.float32)

    # Get the word in each pattern
    words_pattern = docs[0]
    # Lemmatize the word in each pattern
    words_pattern = [lemmatizer.lemmatize(word.lower()) for word in words_pattern]

    # Get the tag in each pattern
    tag = docs[1]

    # create the bag of words array with 1, if word is found in current pattern
    for word in words_pattern:
        if word in words:
            bag_of_words[words.index(word)] = 1
        else :
            bag_of_words[words.index(word)] = 0

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    tag_pattern[tags.index(tag)] = 1

    training.append([bag_of_words, tag_pattern])

#Shuffle the training data (features)
random.shuffle(training)
training = np.array(training, dtype='object')

training[:, 0][:3]

list(training[:, 0][:3])

[array([1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 1.], dtype=float32),
 array([1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 1.], dtype=float32),
 array([1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 1.], dtype=float32)]

In [ ]:
# @title
# create a new dimension for the x_train and y_train

x_train = np.array(list(training[:,0]))
y_train = np.array(list(training[:,1]))

# Convert to Tensor
x_train = tf.convert_to_tensor(x_train)
y_train = tf.convert_to_tensor(y_train)

In [ ]:
# @title
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD # Import the SGD optimizer


# deep neural networds model
model = Sequential()

model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#Training and saving the model

hist = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.1327 - loss: 2.5848
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2070 - loss: 2.4988 
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1199 - loss: 2.5374      
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1603 - loss: 2.4399  
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2046 - loss: 2.4663
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1809 - loss: 2.4246
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2948 - loss: 2.2166
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2219 - loss: 2.2668  
Epoch 9/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1309 - loss: 2.3014
Epoch 10/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2559 - loss: 2.2178  
Epoch 11/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2638 - loss: 2.1390
Epoch 12/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.50

In [ ]:
# @title
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)

# tf.keras.models.save_model(model, 'chatbot_model.h5', save_format='h5')
#model.save('/content/drive/My Drive/chatbot_model.keras')

model.save('chatbot_model.keras')

In [ ]:
# @title
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           2,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 13)                  │             845 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,789 (46.05 KB)

 Trainable params: 11,789 (46.05 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# @title
def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


def bag_of_words(sentence, showDetails=True):
    # Open the word collection from the dataset
    words_collection = pickle.load(open('words.pkl', 'rb'))

    count = 0
    words = []

    # Clear the Sentence (lower case and Lemmatize)
    sentence_split = clean_sentence(sentence)

    # Initialize Token
    tokens = np.zeros(shape=(len(words_collection)), dtype=np.float32)

    # Check if there is a word found on the Word collection
    for word in sentence_split:
        if word in words_collection:
            tokens[words_collection.index(word)] = 1
            words.append(word)
            count += 1

    if showDetails:
        print(f"Found : {count} word(s) on the sentence")
        print(words)

    return tokens



def predict(model, sentence):
    inputs = bag_of_words(sentence, showDetails=False)
    inputs = tf.expand_dims(inputs, axis=0)

    pred = model.predict(inputs)

    pred = tf.squeeze(pred)


    result = tf.math.argmax(pred)

    intent = tags[result]

    return intent, tf.reduce_max(pred)*100


In [ ]:
# @title
import json
import random
import tensorflow as tf
from keras.models import load_model
import os

model = load_model('chatbot_model.keras')

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer= tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)

if os.path.exists('temp_data.json'):
  intents = json.load(open(temp_file_path))
else:
  intents = dataset

words = pickle.load(open('words.pkl','rb'))
#print("Loaded words:", words)
classes = pickle.load(open('tags.pkl','rb'))
#print("Loaded tags:", classes)

def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)

    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    #print("Cleaned sentence words:", sentence_words)

    return sentence_words


def bag_of_words(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)

    for s in sentence_words:
        if s in words:
            index = words.index(s)  # Find the index of the word in the vocabulary
            bag[index] = 1

    return np.array(bag)


def predict_class(sentence):
    p = bag_of_words(sentence, words, show_details=True)
    res = model.predict(np.array([p]))[0]
   # print("Raw predictions:", res)
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    # Print index to tag mapping
   # print("Classes (tags):", classes)

    return_list = []
    for r in results:
        tag = classes[r[0]]
        probability = str(r[1])
        return_list.append({"intent": tag, "probability": probability})

    return return_list

poems = [
    "Two roads diverged in a yellow wood, \nAnd sorry I could not travel both \nAnd be one traveler, long I stood...",
    "I wandered lonely as a cloud \nThat floats on high o'er vales and hills, \nWhen all at once I saw a crowd...",
    "Shall I compare thee to a summer's day? \nThou art more lovely and more temperate...",
    "Hope is the thing with feathers \nThat perches in the soul, \nAnd sings the tune without the words...",
]

def getResponse(ints, intents_json):

    if len(ints) == 0:
      return random.choice(poems)
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']



    for i in list_of_intents:
        if i['tag'] == tag:
            # Randomly select 1
            result = random.choice(i['responses'])
           # result = random.sample(i['responses'], min(3, len(i['responses'])))
            break

    return result


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 8 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# @title EDGAR!!
import json
import tempfile


def load_intents(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def save_intents(file_path, data):
    # Save the data to the specified file path in Colab
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
    print(f"Data saved to {file_path}")

def add_intent(file_path, tag, patterns, responses):

    intents = load_intents(temp_file_path)
#Prefix the tag with "tag:"
 #   prefixed_tag = f"tag:{tag}"

    # Find if the tag already exists in the intents list
    if any(intent['tag'] == tag for intent in intents['intents']): #Iterate over the list of intents
        print(f"Tag '{tag}' already exists.")
    existing_intent = next((intent for intent in intents['intents'] if intent['tag'] == tag), None)

    if existing_intent:
        print("EDGAR: Theme already exists. Updating emotions and poems.")
        existing_intent['patterns'].extend(patterns)
        existing_intent['responses'].extend(responses)
    else:
        print("EDGAR: New poem added. Anything else?")
        new_intent = {'tag': tag, 'patterns': patterns, 'responses': responses}
        intents['intents'].append(new_intent)

    save_intents(file_path, intents)

def check_duplicate(tag, patterns, responses, intents):
    prefixed_tag = f"tag:{tag}"
    existing_intent = next((intent for intent in intents['intents'] if intent['tag'] == tag), None)

    if existing_intent:
        existing_patterns = set(existing_intent['patterns'])
        existing_responses = set(existing_intent['responses'])

        if any(pattern in existing_patterns for pattern in patterns):
            print("EDGAR: One or more patterns already exist for this emotion.")
            return True
        if any(response in existing_responses for response in responses):
            print("EDGAR: One or more poems already exist for this emotion.")
            return True

    return False

flag = True
print("EDGAR: Hi there. My name is Edgar. How are you feeling? (say 'quit' to leave) (If you want to add a poem, say 'add'!)")

while flag:
    user_response = input().lower()

    if user_response != 'quit':
        if user_response == 'add':
            # Get the intents again
          #  intents_file = dataset
           # intents = load_intents(intents_file)
            #intents = dataset.get('intents', [])
            # Get the new intent details
            # Get the intents file path from the user
           # intents_file = '/content/drive/MyDrive/PoetryIntents1234.json'
            #intents = load_intents(intents_file)

            #intents = dataset


            new_tag = input("EDGAR: What emotion does this poem make you feel? (one word!)").strip().lower()
            if new_tag == 'quit':
                continue

            new_patterns = []
            while True:
                pattern = input("EDGAR: How do you feel when reading this poem?").strip().lower()
                new_patterns.append(pattern)
                break


            new_responses = []
            poem = ""
            while True:
                response = input("EDGAR: Please type in the poem, line by line (type 'quit' when finished): ").strip()
                if response.lower() == 'quit':
                    break
                poem += response + "\n"  # Add each line to the poem string, followed by a newline

            if poem.strip():  # Only add if the poem is not empty
                new_responses.append(poem.strip())  # Strip any trailing newlines

            if not check_duplicate(new_tag, new_patterns, new_responses, intents):
                # Add the new intent
                add_intent(temp_file_path, new_tag, new_patterns, new_responses)
            else:
                print("EDGAR: Duplicate entry detected. No changes made.")

        elif user_response in ['thanks', 'thank you']:
            flag = False
            print("EDGAR: You're welcome.")

        else:
            ints = predict_class(user_response)
            res = getResponse(ints, intents)
            print("EDGAR: ", res)

    else:
        flag = False
        print("EDGAR: Goodbye! Take care <3 ")


EDGAR: Hi there. My name is Edgar. How are you feeling? (say 'quit' to leave) (If you want to add a poem, say 'add'!)


KeyboardInterrupt: Interrupted by user

In [ ]:
# @title Code to download the file
from google.colab import files

# Download the file
files.download(temp_file_path)
